In [1]:
import os 
import sys
import numpy as np
import argparse
from scipy.spatial.transform import Rotation as R 
# add catkin_ws context 
sys.path.append("/home/junting/franka_ws/devel/lib/python3.9/site-packages")

from src.lmp import *
from src.env.true_grounding_env import TrueGroundingEnv
from src.config import cfg_tabletop
import rospy 
import rospkg
import jupyros as jr

from std_msgs.msg import String, Header
from geometry_msgs.msg import PoseStamped, Pose, Point, Quaternion


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
########################################
# initialize environment
########################################
rospy.init_node('eval_code', log_level=rospy.DEBUG)
# get package root path 
pkg_root = rospkg.RosPack().get_path('instruct_to_policy')

# setup environment
env = TrueGroundingEnv(cfg_tabletop)
env.reset()


[DEBUG] [1699098719.186669, 0.000000]: init_node, name[/eval_code], pid[666986]
[DEBUG] [1699098719.189006, 0.000000]: binding to 0.0.0.0 0
[DEBUG] [1699098719.190568, 0.000000]: bound to 0.0.0.0 38251
[DEBUG] [1699098719.200649, 0.000000]: connecting to junting-shlab 44139
[DEBUG] [1699098719.201550, 0.000000]: ... service URL is rosrpc://junting-shlab:38251
[DEBUG] [1699098719.203015, 0.000000]: [/eval_code/get_loggers]: new Service instance
[DEBUG] [1699098719.204851, 0.000000]: ... service URL is rosrpc://junting-shlab:38251
[DEBUG] [1699098719.205905, 0.000000]: [/eval_code/set_logger_level]: new Service instance
[INFO] [1699098719.489823, 261.652000]: camera_left: Waiting for camera_left/color/camera_info...
[DEBUG] [1699098719.506182, 261.660000]: connecting to junting-shlab 44139
[DEBUG] [1699098719.529059, 261.669000]: connecting to junting-shlab 51171
[DEBUG] [1699098719.543127, 261.676000]: connecting to junting-shlab 44139
[DEBUG] [1699098719.546820, 261.677000]: connecting

In [3]:
# Define parameters 
# processed_file = rospy.get_param('~processed_file', 'data/table_cabinet_100/generated_code/processed_table_cabinet_4.json')
# raw_file = rospy.get_param('~raw_file', 'data/table_cabinet_100/generated_code/raw_table_cabinet_4.json')
processed_file = rospy.get_param('~processed_file', 'data/benchmark/generated_code/processed_world_1_table_sort.json')
raw_file = rospy.get_param('~raw_file', 'data/benchmark/generated_code/raw_world_1_table_sort.json')


In [ ]:
# Load files & data 
def filter_drawer0(processed_data, raw_data, mask):
    """
    Filter tasks involving interacting with drawer 0, since it is too far from the robot.
    """

    for i, data in enumerate(processed_data):
        if mask[i] == 0: # already filtered in previous filters 
            continue
        full_query = data['query']
        instruction = full_query.split(';')[-1]
        if 'drawer0' in instruction:
            mask[i] = 0
            
    return mask 


def filter_tasks(processed_data, raw_data):
    """
    Filter the tasks that are not suitable for the environment based on hand-crafted rules.
    """
    filter_funcs = [filter_drawer0]
    mask = np.ones((len(processed_data)))
    for func in filter_funcs:
        mask = func(processed_data, raw_data, mask)
        
    filtered_processed_data = [processed_data[i] for i in range(len(processed_data)) if mask[i] == 1]
    filtered_raw_data = [raw_data[i] for i in range(len(raw_data)) if mask[i] == 1]
    return filtered_processed_data, filtered_raw_data

processed_file_path = os.path.join(pkg_root, processed_file)
raw_file_path = os.path.join(pkg_root, raw_file)

with open(processed_file_path, 'r') as f:
    processed_data = json.load(f)
with open(raw_file_path, 'r') as f:
    raw_data = json.load(f)

# filter tasks that are not suitable for the environment
filtered_processed_data, filtered_raw_data = filter_tasks(processed_data, raw_data)

## Pick and Place

In [ ]:
env.reset()

[DEBUG] [1698656737.439594, 274.971000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-31-274.971)
[DEBUG] [1698656737.441814, 274.973000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-31-274.971)
[DEBUG] [1698656737.443733, 274.975000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-31-274.971)
[DEBUG] [1698656737.458981, 274.984000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-32-274.981)
[DEBUG] [1698656737.498108, 275.018000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-32-274.981)
[DEBUG] [1698656737.499845, 275.019000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-32-274.981)
[DEBUG] [1698656737.503200, 275.022000]: connecting to junting-shlab 49819
[DEBUG] [1698656737.509374, 275.025000]: connecting to junting-shlab 57077
[INFO] [1698656740.276883, 276.026000]: Environment reset.


In [ ]:
pose_msg = env.parse_grasp_pose("apple")

[DEBUG] [1698656740.334489, 276.072000]: connecting to junting-shlab 49819
[DEBUG] [1698656740.354981, 276.084000]: connecting to junting-shlab 49819
[INFO] [1698656740.420285, 276.119000]: Sending perception data to grasp detection service
[DEBUG] [1698656740.424714, 276.132000]: connecting to junting-shlab 37257


In [ ]:
rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)

In [ ]:
env.open_gripper()

[DEBUG] [1698656747.812783, 282.653000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-33-282.652)
[DEBUG] [1698656747.871143, 282.690000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-33-282.652)
[DEBUG] [1698656747.873134, 282.691000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-33-282.652)


True

In [ ]:
env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)

[DEBUG] [1698656748.031962, 282.788000]: Waiting for rviz to update
[DEBUG] [1698656749.102417, 283.788000]: Waiting for rviz to update


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  points: 
    - 
      positions: [-0.0001196386968160823, -0.785465256688334, 1.8270755118621196e-05, -2.3560126624341233, -4.327726983621716e-05, 1.5712068961231145, 0.7853328772094148]
      velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      accelerations: [1.192689504800483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [0.016277270507500236, -0.7759747046619727, -0.020890949924227614, -2.3653395344690615, -0.014599861679874575, 1.583789890237532, 0.7179472770848913]
      velocities: [0.16624469896876343, 0.09932673633674126, -0.21584982760525312, -0.09197566486582076, -0.15128010017450377, 0.1286837304243048, -0.6949532293251182]
 

In [ ]:
env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)

[DEBUG] [1698656752.367026, 286.748000]: Waiting for rviz to update
[DEBUG] [1698656753.443336, 287.751000]: Waiting for rviz to update


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  points: 
    - 
      positions: [0.5601450325168198, -0.06835683998728648, -0.9763481665279103, -2.4653945969206577, -0.3664473012040599, 2.4530681925183266, -2.4043627185127754]
      velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      accelerations: [0.21304158255930042, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [0.5693778931064011, 0.009400249558381465, -0.9943535626527703, -2.4784654452937187, -0.2872363264524831, 2.5244673435008704, -2.4621138869689694]
      velocities: [-0.00885783775044955, 0.45739518645530197, -0.010678072800921968, -0.05843169345407202, 0.5426137675096455, 0.38160566538445795, -0.3918282034828328]
      ac

In [ ]:
env.close_gripper(width=0.05, force=30)

[DEBUG] [1698656756.172090, 290.190000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-34-290.185)
[DEBUG] [1698656756.384911, 290.358000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-34-290.185)
[DEBUG] [1698656756.387223, 290.358000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-34-290.185)


True

In [ ]:
env.attach_object("apple")

In [ ]:
place_pose = env.parse_place_pose(object_name="apple", receptacle_name="white_ceramic_plate")

[DEBUG] [1698656756.544129, 290.439000]: connecting to junting-shlab 49819
[DEBUG] [1698656756.554554, 290.440000]: connecting to junting-shlab 49819
[DEBUG] [1698656756.594827, 290.467000]: connecting to junting-shlab 49819
[DEBUG] [1698656756.601341, 290.469000]: connecting to junting-shlab 49819


In [ ]:
env.move_to_pose(place_pose)
env.open_gripper()

[DEBUG] [1698656756.640624, 290.492000]: Waiting for rviz to update


[WARN] [1698656758.251163, 291.538000]: Could not plan cartesian_path to target pose 
position: 
  x: -0.150848999999265
  y: 0.28277600000003644
  z: 1.214059999953244
orientation: 
  x: 0
  y: 1
  z: 0
  w: 0.
 Plan accuracy: 0.0


[ INFO] [1698656766.238760997, 297.136000000]: ABORTED: CONTROL_FAILED
[DEBUG] [1698656766.242013, 297.138000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-35-297.137)
[DEBUG] [1698656766.246857, 297.142000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-35-297.137)
[DEBUG] [1698656766.248159, 297.143000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-35-297.137)


[WARN] [1698656766.239073, 297.136000]: Execution failed! Going to retry with error recovery


[DEBUG] [1698656767.448655, 297.940000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-36-297.939)
[DEBUG] [1698656767.652939, 298.074000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-36-297.939)
[DEBUG] [1698656767.654454, 298.075000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-36-297.939)


True

In [ ]:
env.detach_object("apple")

## Open Drawer

In [ ]:
jr.publish('/rviz/moveit/move_marker/goal_panda_hand_tcp', PoseStamped)

In [39]:
env.reset()
# env.open_gripper()

[DEBUG] [1699102252.106293, 2021.942000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-18-2021.940)
[DEBUG] [1699102252.108617, 2021.942000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-18-2021.940)
[DEBUG] [1699102252.110524, 2021.943000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-18-2021.940)
[DEBUG] [1699102274.347113, 2030.940000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-19-2030.940)
[DEBUG] [1699102278.660053, 2032.807000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-19-2030.940)
[DEBUG] [1699102278.662155, 2032.807000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-19-2030.940)
[DEBUG] [1699102278.674807, 2032.817000]: connecting to junting-shlab 44139
[DEBUG] [1699102278.702581, 2032.827000]: connecting to junting-shlab 55529
[INFO] [1699102283.799407, 2033.828000]: Environment reset.


In [27]:
# env.get_obj_name_list()
# [bbox.object_id for bbox in env.gazebo_gt_bboxes]
env.get_3d_bbox("cabinet.handle_0")

array([-1.07537923, -0.20869142,  1.44199707, -1.01066928,  0.19866786,
        1.4639238 ])

In [40]:
grasp_pose = env.parse_grasp_pose(object_name="cabinet.handle_0")
print(grasp_pose)

[INFO] [1699102348.234143, 2068.867000]: Sending perception data to grasp detection service
[DEBUG] [1699102348.248152, 2068.873000]: connecting to junting-shlab 38187
position: 
  x: -0.8817051649093628
  y: 0.15365920960903168
  z: 1.4527581930160522
orientation: 
  x: -0.4370810435130189
  y: -0.5328791379354131
  z: 0.5602237696813563
  w: 0.459509862395566


In [41]:
env.publish_goal_to_marker(grasp_pose)

[DEBUG] [1699102392.500664, 2092.432000]: Waiting for rviz to update


In [42]:
env.grasp(grasp_pose)
# env.move_to_pose(grasp_pose)

[DEBUG] [1699102404.536568, 2098.647000]: Waiting for rviz to update


[WARN] [1699102406.564774, 2099.684000]: Could not plan cartesian_path to target pose 
position: 
  x: -0.7837598830695041
  y: 0.17319691146470184
  z: 1.447758195865096
orientation: 
  x: -0.4370810435130189
  y: -0.5328791379354131
  z: 0.5602237696813563
  w: 0.459509862395566.
 Plan accuracy: 0.7058823529411765


[DEBUG] [1699102416.952742, 2105.004000]: Waiting for rviz to update


True

In [43]:
env.close_gripper()

[DEBUG] [1699102428.506013, 2111.240000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-20-2111.240)


KeyboardInterrupt: 

In [44]:
# generate a horizontal trajectory to open the drawer
grasp_position = np.array([grasp_pose.position.x, grasp_pose.position.y, grasp_pose.position.z])
pull_position = grasp_position + np.array([0.2, 0, 0]).astype(float)
pull_pose = Pose(position=Point(*pull_position), orientation=grasp_pose.orientation)

In [45]:
env.move_to_pose(pull_pose)

[DEBUG] [1699102458.032604, 2126.021000]: Waiting for rviz to update
[DEBUG] [1699102460.685355, 2127.408000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-20-2111.240)
[DEBUG] [1699102460.687381, 2127.409000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-20-2111.240)


joint_trajectory: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  points: 
    - 
      positions: [1.4391089408157933, -1.1595114729967637, 1.083618030513942, -2.8203575968691688, 2.5978514888838067, 1.5232491083055946, 0.5389553680635224]
      velocities: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      accelerations: [1.0061420142808317, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
      effort: []
      time_from_start: 
        secs: 0
        nsecs:         0
    - 
      positions: [1.456085767463846, -1.1818373289352964, 1.0427637554380667, -2.8697534074208955, 2.568984452910666, 1.4908000986335141, 0.5441056330476423]
      velocities: [0.1271249385934141, -0.2190416552789484, -0.38346214938805595, -0.44316420918841337, -0.28605165783012715, -0.30764815566426806, 0.05206359174902922]
      accelerations:

In [ ]:
env.open_gripper()

[DEBUG] [1698667486.144047, 2393.055000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-4-2393.055)


KeyboardInterrupt: 

In [ ]:
pose_msg = env.parse_grasp_pose(object_name="apple")

rot_mat = R.from_quat([pose_msg.orientation.x, pose_msg.orientation.y, pose_msg.orientation.z, pose_msg.orientation.w]).as_matrix()
translation = np.array([pose_msg.position.x, pose_msg.position.y, pose_msg.position.z])

depth = 0.05

pregrasp_offset_local = np.array([0, 0, -0.15]).astype(np.float32)
# predicted gripper center is 0.02m above the gripper tip
approach_offset_local = np.array([0, 0, depth - 0.02 ]).astype(np.float32)
pregrasp_position = translation + rot_mat @ pregrasp_offset_local
approach_position = translation + rot_mat @ approach_offset_local

pregrasp_pose = Pose(position=Point(*pregrasp_position), orientation=pose_msg.orientation)
approach_pose = Pose(position=Point(*approach_position), orientation=pose_msg.orientation)
 
env.open_gripper()
 
# env.publish_goal_to_marker(pregrasp_pose)
env.move_to_pose(pregrasp_pose)
# env.publish_goal_to_marker(approach_pose)
env.move_to_pose(approach_pose)
env.close_gripper(width=0.05, force=30)
env.attach_object("apple")


[DEBUG] [1698667803.594753, 2503.881000]: connecting to junting-shlab 51369
[DEBUG] [1698667803.622338, 2503.901000]: connecting to junting-shlab 51369
[INFO] [1698667803.689111, 2503.937000]: Sending perception data to grasp detection service
[DEBUG] [1698667803.692310, 2503.939000]: connecting to junting-shlab 42479
[DEBUG] [1698667812.567327, 2509.665000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-8-2509.664)
[DEBUG] [1698667812.624500, 2509.699000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-8-2509.664)
[DEBUG] [1698667812.626343, 2509.699000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-8-2509.664)
[DEBUG] [1698667812.628136, 2509.699000]: Waiting for rviz to update
[DEBUG] [1698667816.583975, 2512.567000]: Waiting for rviz to update
[DEBUG] [1698667819.731957, 2514.969000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-9-2514.967)
[DEBUG] [1698667820.049443, 2515.169000]: Transitioning 

In [ ]:
env.reset()

[DEBUG] [1698668123.212258, 2736.405000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-10-2736.405)
[DEBUG] [1698668123.214196, 2736.407000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-10-2736.405)
[DEBUG] [1698668123.216571, 2736.408000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-10-2736.405)
[DEBUG] [1698668123.225734, 2736.413000]: Transitioning to ACTIVE (from WAITING_FOR_GOAL_ACK, goal: /eval_code-11-2736.412)
[DEBUG] [1698668125.570685, 2738.318000]: Transitioning to WAITING_FOR_RESULT (from ACTIVE, goal: /eval_code-11-2736.412)
[DEBUG] [1698668125.573065, 2738.319000]: Transitioning to DONE (from WAITING_FOR_RESULT, goal: /eval_code-11-2736.412)
[DEBUG] [1698668125.590189, 2738.330000]: connecting to junting-shlab 51369
[DEBUG] [1698668125.598430, 2738.331000]: connecting to junting-shlab 48309
[INFO] [1698668128.794415, 2739.354000]: Environment reset.


[DEBUG] [1698668283.938680, 2868.541000]: [/tf_static] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1698668283.971014, 2868.566000]: [/tf] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1698668283.974217, 2868.568000]: [/tf_static] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1698668284.061461, 2868.614000]: Unable to connect subscriber to publisher [http://junting-shlab:45991/] for topic [/tf_static]: [Errno 111] Connection refused
[DEBUG] [1698668284.094302, 2868.629000]: [/camera_left/aligned_depth/image_raw] failed to receive incoming message : unable to receive data from sender, check sender's logs for details
[DEBUG] [1698668284.132709, 2868.650000]: [/camera_right/aligned_depth/image_raw] failed to receive incoming message : unable to receive data from sender, check sender's lo